<a href="https://colab.research.google.com/github/evsemenova/Python-for-Analysts/blob/HW_5/%D0%A3%D1%80%D0%BE%D0%BA_5_%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0_%D0%B2_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B5_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%D0%B0_%D1%81_%D0%B1%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%BE%D0%B9_Scipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Новый раздел

**1. Взять датасет из домашнего задания №2. Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и значимо ли это отличие статистически.**

In [38]:
#Импортируем нужные библиотеки
import pandas as pd
import scipy.stats as stats
import numpy as np
from scipy.optimize import Bounds
from scipy.optimize import minimize

In [39]:
df_marketing = pd.read_csv('marketing_campaign.csv')

In [40]:
df_subscribers = pd.read_csv('subscribers.csv')

In [41]:
df_users = pd.read_csv('users.csv')

In [42]:
df_marketing.head(1)

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True


In [43]:
df_subscribers.head(1)

,user_id,subscribing_channel,date_subscribed,date_canceled,is_retained
0,a1000,House Ads,1/1/18,NaN,True


In [44]:
df_users.head(1)

,user_id,age_group,language_preferred
0,a1000,0-18 years,English


In [45]:
# Объеденим таблицы
df_new = df_marketing.merge(df_subscribers, on='user_id', how='inner').merge(df_users, on='user_id', how='inner')

In [46]:
df_new.head(1)

,user_id,date_served,marketing_channel,variant,language_displayed,converted,subscribing_channel,date_subscribed,date_canceled,is_retained,age_group,language_preferred
0,a1000,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True,0-18 years,English


In [47]:
# Создадим таблицу сопряженности
df_conjugacy = df.groupby('variant')['converted'].value_counts().unstack()
df_conjugacy.head()




converted,False,True
variant,,
control,4705,371
personalization,4241,705


In [48]:
# определяем значимость аб-теста
result = stats.chi2_contingency(df_conjugacy, correction = False)
chisq, pvalue = result[:2]
print('chisq = {}, pvalue = {}'.format(chisq, pvalue))

chisq = 126.07767827548582, pvalue = 2.9568009705199523e-29


В personalization конверсия выше.

**2. Цех может производить стулья и столы. На производство стула идет 5 единиц материала, на производство стола - 20 единиц (футов красного дерева). Стул требует 10 человеко-часов, стол - 15. Имеется 400 единиц материала и 450 человеко-часов. Прибыль при производстве стула - 45 долларов США, при производстве стола - 80 долларов США. Сколько надо сделать стульев и столов, чтобы получить максимальную прибыль?**

In [58]:
# Задаем целевую функцию:
# Компанией производится два вида продукции:
# x0 — стул, прибыль 45 $ 
# x1 — стол, прибыль 80 $

def value(x):
    return - 45*x[0] - 80*x[1]


In [59]:
# Задаем систему ограничений:
# Фонд их рабочего времени на месяц - 450 человеко-часов
# Наличие материалов - 400 ед
# На изготовление стула тратится: 10 часов и 5 ед материла
# На изготовление стола тратится: 15 часов и 20 ед материала
neq_cons = {'type': 'ineq',
             'fun': lambda x: np.array ([450 - 10 * x [0] - 15 * x [1],
                                         400 - 5 * x [0] - 20 * x [1]])}


In [60]:
# Ограничение — выпуск продукции должен быть только положительным:
from scipy.optimize import Bounds
bnds = Bounds([0, 0], [np.inf, np.inf, np.inf])

In [61]:
# Максимизируем функцию ежемесячной прибыли
x0 = np.array([10, 10])
res = minimize(value, x0, method='SLSQP', constraints=neq_cons, bounds=bnds)
print(res.x)


[24. 14.]


Для получения max прибыли необходимо произвести 24 стула и 14 столов
1080 
